# Raw Processing

## Preparation


In [1]:
import pandas as pd
from random import choice
from datetime import datetime

pd.set_option('display.max_columns', None)

## Data Information & Description

In [2]:
# sample from a random year
year = choice([i for i in range(2000, 2010)])
df = pd.read_parquet(f'../data/raw/transfers_by_day/y={year}/', engine='pyarrow')
df['y'] = str(year)
df.sample(2)

,portrait_url,name,player_id,position,age,nationalities,left_club_url,left_club_name,left_club_name_alt,left_club_league_country_url,left_club_league_country_name,left_club_league_name,left_club_league_name_alt,join_club_url,join_club_name,join_club_name_alt,join_club_league_country_url,join_club_league_country_name,join_club_league_name,join_club_league_name_alt,market_value,fee,loan_fee,transfer_url,transfer_date,ingested_at,m,d,y
60883,https://img.a.transfermarkt.technology/portrai...,Alexis,86295,Centre-Forward,26,"[{""name"": ""Portugal"", ""url"": ""https://tmssl.ak...",https://tmssl.akamaized.net/images/wappen/tiny...,Beira-Mar,SC Beira-Mar,https://tmssl.akamaized.net/images/flagge/tiny...,Portugal,Liga Sabseg,Liga Sabseg,https://tmssl.akamaized.net/images/wappen/tiny...,Oliveira Bairro,Oliveira do Bairro SC,https://tmssl.akamaized.net/images/flagge/very...,Portugal,,,€150k,loan transfer,,/jumplist/transfers/spieler/86295/transfer_id/...,2009-07-01,2023-03-05 01:18:00,7,1,2009
52433,https://img.a.transfermarkt.technology/portrai...,Fabio Roselli,29567,Defensive Midfield,25,"[{""name"": ""Italy"", ""url"": ""https://tmssl.akama...",https://tmssl.akamaized.net/images/wappen/tiny...,Legnano,Legnano AC,https://tmssl.akamaized.net/images/flagge/tiny...,Italy,Serie C - A,Serie C - A,https://tmssl.akamaized.net/images/wappen/tiny...,Ravenna,Ravenna FC,https://tmssl.akamaized.net/images/flagge/very...,Italy,Serie C - A,Serie C - A,€300k,free transfer,,/jumplist/transfers/spieler/29567/transfer_id/...,2009-01-01,2023-03-03 20:12:54,1,1,2009


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23708 entries, 51216 to 74904
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   portrait_url                   23708 non-null  object  
 1   name                           23708 non-null  object  
 2   player_id                      23708 non-null  object  
 3   position                       23708 non-null  object  
 4   age                            23708 non-null  object  
 5   nationalities                  23708 non-null  object  
 6   left_club_url                  23708 non-null  object  
 7   left_club_name                 23708 non-null  object  
 8   left_club_name_alt             23708 non-null  object  
 9   left_club_league_country_url   23708 non-null  object  
 10  left_club_league_country_name  23708 non-null  object  
 11  left_club_league_name          23708 non-null  object  
 12  left_club_league_name_alt   

## Create Clean Raw Data

The clean raw data processing is for removing duplicates and merge splitted parquet files of a transfer date (y/m/d)

In [5]:
total_rows = 0
total_non_duplicated_rows = 0
total_duplicated_rows = 0
ymdhms = datetime.now().strftime('%Y%m%d%H%M%S')

for year in range(2000, 2016):
    df = pd.read_parquet(f'../data/raw/transfers_by_day/y={year}/', engine='pyarrow')
    df['y'] = str(year)
    duplicated = df.duplicated(subset=['player_id', 'transfer_date']).value_counts()
    total_rows = total_rows + (df.shape[0] if len(df.shape) >= 1 else 0)
    total_non_duplicated_rows = total_non_duplicated_rows + (duplicated[0] if len(duplicated) >= 1 else 0)
    total_duplicated_rows = total_duplicated_rows + ((duplicated[1] if len(duplicated) >= 2 else 0))
    
    # remove duplicated
    df.drop_duplicates(subset=['player_id', 'transfer_date'], inplace=True)
    
    # write to a new dataset
    df.to_parquet(f'../data/raw/transfers_by_day_clean_{ymdhms}/', partition_cols=['y', 'm', 'd'])
    
print(f"total_rows: {total_rows}")
print(f"total_non_duplicated_rows: {total_non_duplicated_rows}")
print(f"total_duplicated_rows: {total_duplicated_rows}")
    

total_rows: 308393
total_non_duplicated_rows: 307367
total_duplicated_rows: 1026
